In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt

from gprm import ReconstructionModel
from gprm.datasets import Rocks, Reconstructions, Paleogeography, Geology
from gprm.utils.raster import to_anchor_plate
from gprm.utils.geometry import nearest_feature

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../../andes_paper/python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap
from time_series import lowess

import collections

%matplotlib inline
%load_ext autoreload
%autoreload 2


########## Geochemistry Inputs
#df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv')
df = joy.geochem_from_csv('../datafiles/geochem_global_20230124.csv',
                          longitude_field_name='longitude', latitude_field_name='latitude')
df = df[df['land_mask']>0.5].reset_index(drop=True)


model_dir = '../../andes_paper/luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)


In [2]:
M2021 = Reconstructions.fetch_Merdith2021()
subduction_lookup_time_step = 1.

sz_lookup = {}
for reconstruction_time in np.arange(0,1001,subduction_lookup_time_step):
    snapshot = M2021.plate_snapshot(reconstruction_time=reconstruction_time)
    szs = snapshot.get_boundary_features(boundary_types=['subduction'])
    sz_lookup[reconstruction_time] = szs


def apply_nearest_feature(point, lookup_dict):
    
    return nearest_feature(pygplates.PointOnSphere(point.geometry.y, point.geometry.x), 
                           lookup_dict[point['AnalysisAge']])*pygplates.Earth.mean_radius_in_kms


In [3]:
# Here, we are only interested in extracting the distance to the nearest subduction zone,
# at the best temporal resolution we reasonable can (ie 1 Myr)
time_bin_size = subduction_lookup_time_step

df = df.query('age>0.1 & age<=1000.')
df['AnalysisAge'] = np.round(df['age']/subduction_lookup_time_step) * subduction_lookup_time_step

df = M2021.assign_plate_ids(df, copy_valid_times=True)

# throw away data that won't get reconstructed properly
df = df.query('age<=FROMAGE')

df = M2021.reconstruct_to_time_of_appearance(df, ReconstructTime='AnalysisAge')

df['distance_to_sz'] = df.apply(lambda x: apply_nearest_feature(x, sz_lookup), axis=1)

df = df.rename(columns={'geometry': 'rgeometry'})

df['geometry'] = gpd.points_from_xy(df.longitude, df.latitude)

df.to_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv', index=False)


In [ ]:
#df['distance_to_sz'].hist(bins=np.arange(0,5000,100))

#df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')

#gs = gpd.GeoSeries.from_wkt(df['geometry'])
#df = gpd.GeoDataFrame(df, geometry=gs)


In [ ]:
df = pd.read_csv('../datafiles/geochem_global_20230124_subduction_M2021.csv')
df = gpd.GeoDataFrame(df, geometry=gpd.GeoSeries.from_wkt(df['geometry']))

time_bin_size = 25

#df = joymap.select_orogens(df,gdf=None, 
#                           orogen_names=None, 
#                           continent_names=[
#                               'Europe'
#                           ]
#                          )

df['bin_age'] = np.round(df['age']/time_bin_size) * time_bin_size

#df = M2021.assign_plate_ids(df, copy_valid_times=True)

df = M2021.reconstruct_to_time_of_appearance(df, ReconstructTime='bin_age')

df.plot()

In [ ]:
calibration = 'luffi'
mohometer_selection = 41

if isinstance(mohometer_selection, list):
    mohometer_description_string = '|'.join(mohometer_selection)
else:
    mohometer_description_string = str(mohometer_selection)
    

df_filt = joy.filter_the_database(df, 
                                  filter_method=calibration,
                                  age_min=0.1, 
                                  age_max=1000.) 


space_bin_size = 2
sz_distance_limit = 750.

df_sel = df_filt[(df_filt['distance_to_sz']<sz_distance_limit)].reset_index(drop=True)

elevation_estimates = joy.get_elevations(df_sel, 
                                         gc_interpolator_dict=gc_interpolator_dict,
                                         calibration=calibration,
                                         mohometer_selection=mohometer_selection)
elevation_estimates = gpd.GeoDataFrame(elevation_estimates.join(df_sel['bin_age']),
                                       geometry=df_sel.geometry)

elevation_estimates['bin_latitude'] = np.round(elevation_estimates.geometry.y/space_bin_size) * space_bin_size
elevation_estimates['bin_longitude'] = np.round(elevation_estimates.geometry.x/space_bin_size) * space_bin_size

age_groups = elevation_estimates.groupby(by=['bin_age'])
age_keys = []
median_elevations = []
elevation_sems = []
nbins_list = []

for name, group in age_groups:
    age_keys.append(name)
    geog_groups = group.groupby(by=['bin_latitude', 'bin_longitude'])
    
    #'''
    fig=pygmt.Figure()
    fig.basemap(frame='afg', projection='N15c', region='d')
    #fig.coast(land='gray', projection='N15c', region='d')
    M2021.polygon_snapshot('continents', name+0.001).plot(fig, fill='grey', pen='0p,gray')
    M2021.plate_snapshot(name).plot_boundaries(fig)
    pygmt.makecpt(cmap='cubhelix', series=[-1000,5000,500], background=True, reverse=True)
    tmp = pd.DataFrame(data={'x':geog_groups.bin_longitude.median().tolist(),
                             'y':geog_groups.bin_latitude.median().tolist(),
                             'z':geog_groups.la_yb_elevation.median().tolist()}).dropna(subset='z')
    #fig.plot(x=geog_groups.bin_longitude.median(), y=geog_groups.bin_latitude.median(), fill=geog_groups.la_yb_elevation.median(), 
    #         style='c0.1c', pen='k', cmap=True)
    if len(tmp)>0:
        fig.plot(x=tmp.x, y=tmp.y, fill=tmp.z, 
                 style='c0.2c', pen='0.2p,black', cmap=True)
    fig.savefig('../images/sequence_M2021/time_series_bins_{:0.0f}Ma.png'.format(name))
    #fig.show()
    #'''
    
    binned_list = []
    for n,g in geog_groups:
        # drop every column that isn't an elevation column, and stack the rest in each group
        data = g.drop(columns=['bin_age', 
                               'geometry', 
                               'bin_latitude', 
                               'bin_longitude']).stack()

        # get the stats of all elevation estimates
        binned_list.append([n[1], 
                            n[0], 
                            name,
                            data.median(),
                            data.quantile(q=0.25),
                            data.quantile(q=0.75)])

    binned_df = pd.DataFrame(data=binned_list, 
                             columns=['x', 'y', 'bin_age',
                                      'median_elevation', 
                                      'q25', 'q75'])

    median_elevations.append(binned_df.median_elevation.median())
    elevation_sems.append(binned_df.median_elevation.sem())
    print(name,
          binned_df.median_elevation.median(),
          binned_df.median_elevation.sem(),
          len(tmp))
    #break
    

ind = np.isnan(median_elevations)
res, rese = lowess(np.array(age_keys)[~ind], np.array(median_elevations)[~ind], xwidth=100)
plt.figure(figsize=(5,8))
plt.plot(age_keys, median_elevations,'ro')
plt.plot(np.array(age_keys)[~ind], res, 'k')
plt.fill_between(np.array(age_keys)[~ind], 
                 y1=res-rese, y2=res+rese, alpha=0.2)
plt.ylim(500,4500)
#plt.xlim(0,540)
plt.title('Distance Limit {:0.0f}km | Num Mohometers {:d}'.format(
    sz_distance_limit,
    mohometer_selection))
plt.show()
